# Programación declarativa @ GIA - URJC
## Curso 24-25
## Convocatoria extraordinaria
## Prueba 2

La duración de la prueba es de 1 hora y 20 minutos.


# Preámbulo

In [ ]:
import $ivy.`org.scalatest::scalatest:3.2.16`
import org.scalatest.{Filter => _, _}, flatspec._, matchers._

In [ ]:
object SignaturesList :
    abstract class List[A]:
        
        // Common HOFs
        def foldRight[B](nil: B)(cons: (A, B) => B): B
        def foldLeft[B](initial: B)(update: (B, A) => B): B
        def map[B](f: A => B): List[B]
        def flatMap[B](f: A => List[B]): List[B]
        def filter(f: A => Boolean): List[A]

        // Devuelve cierto si todos los elementos de la lista satisfacen el predicado de entrada.
        def forall(pred: A => Boolean): Boolean
 
        // Devuelve el tamaño (número de elementos) de la lista.
        def size: Int

        // Devuelve una lista formada por todos los pares de elementos que se encuentran en la misma posición
        // de esta lista y la lista `l`. La lista resultante contendrá tantos pares como el tamaño de la lista más pequeña.
        def zip[B](l: List[B]): List[(A, B)]
        

In [ ]:
object SignaturesOption:

    enum Option[+A]:
        case None extends Option[Nothing]
        case Some(a: A)

        // Common HOFs
        def flatMap[B](f: A => Option[B]): Option[B] = 
            this match 
                case None => None
                case Some(a) => f(a)
        
        // Devuelve cierto si esta valor opcional es de tipo `Some`
        def isDefined: Boolean = this match 
            case None => false
            case _ => true

        // Si este valor opcional es un `Some` devuelve su valor, en caso contrario, se producirá una excepción.
        def get: A = (this : @unchecked) match
            case Some(a) => a


In [ ]:
object SignaturesEither:

    enum Either[A, B]:
        case Left(a: A)
        case Right(b: B)


# Ejercicio 1 (3 puntos)

Supóngase que se desea implementar una función que, dada una cadena de caracteres, transforma dicha cadena en un número, comprueba que es distinto de cero y devuelve su inversa. En caso de que la cadena no represente un número, la función deberá devolver un error. Igualmente, si la cadena sí representa un número pero es cero, también se devolverá un error.

La gestión de errores mediante excepciones nos lleva a programar esta función de la siguiente forma: 

In [ ]:
def inverseOf_Impure(s: String): Double = 
    val i: Int = s.toInt ; 
    assert(i != 0) ; 
    1/i

En este caso, tanto si la cadena no es número como si es cero, el programa lanzará una excepción. En la signatura de la función no se refleja en absoluto esta posibilidad, de ahí que caracterizemos a esta función como una función _impura_. 

Alternativamente al uso de excepciones, podemos utilizar valores opcionales para gestionar los posibles errores mediante una función _pura_: 

In [ ]:
def inverseOf_Pure(s: String): Option[Double] = 
    ???

El propósito de este ejercicio es realizar una implementación de esta función pura análoga funcionalmente a la versión impura mostrada anteriormente.

#### a) (1 punto)

Implementa las siguientes funciones auxiliares `assertNonZero` y `inverse`. La función `assertNonZero` recibe un entero y devuelve `None` en caso de que sea cero; en caso contrario, devuelve el valor de entrada como parte de un valor de tipo `Some`.

In [ ]:
class TestNonZero(nonZero: Int => Option[Int]) extends AnyFlatSpec with should.Matchers:

    "nonZero" should "work" in:
        nonZero(22) shouldBe Some(22)
        nonZero(1) shouldBe Some(1)
        nonZero(0) shouldBe None

In [ ]:
// IMPLEMENTA TU RESPUESTA

def assertNonZero(i: Int): Option[Int] = 
    if i == 0 then None
    else Some(i)

In [ ]:
run(TestNonZero(assertNonZero))

La función `inverse` devuelve el inverso del valor de entrada como parte de un valor de tipo `Some`, sin comprobar previamente si dicho valor es cero o no. 

In [ ]:
class TestInverse(inverse: Int => Option[Double]) extends AnyFlatSpec with should.Matchers:

    "inverse" should "work" in:
        inverse(1) shouldBe Some(1/1)
        inverse(4) shouldBe Some(1/4)
        inverse(8) shouldBe Some(1/8)

In [ ]:
// IMPLEMENTA TU RESPUESTA

def inverse(i: Int): Option[Double] = 
    Some(1/i)

In [ ]:
run(TestInverse(inverse))

#### b) (1 punto)

Para implementar la función `inverseOf` se hará uso de las funciones del apartado anterior, así como del método `toIntOption` de la clase `Int`, que parsea una cadena de caracteres en un número, devolviendo el resultado en un valor opcional. Por ejemplo:

In [ ]:
"123".toIntOption == Some(123)
"X".toIntOption == None

Algunos ejemplos de uso de la función `inverseOf` son los siguientes:

In [ ]:
class TestInverseOf(inverseOf: String => Option[Double]) extends AnyFlatSpec with should.Matchers:

    "inverseOf" should "work" in:
        inverseOf("4") shouldBe Some(1/4)
        inverseOf("1") shouldBe Some(1/1)
        inverseOf("X") shouldBe None
        inverseOf("0") shouldBe None

Implementa la función `inverseOf` _sin_ utilizar funciones de orden superior.

In [ ]:
// IMPLEMENTA TU RESPUESTA

def inverseOf(s: String): Option[Double] = 
    s.toIntOption match 
        case None => None
        case Some(i) =>
            assertNonZero(i) match 
                case None => None
                case Some(i) => 
                    inverse(i)

In [ ]:
run(TestInverseOf(inverseOf))

#### c) (1 punto)

Implementa la función `inverseOf` utilizando la función de orden superior `flatMap` para valores de tipo `Option[_]` (véase la definición de esta función en el preámbulo del examen).

In [ ]:
// IMPLEMENTA TU RESPUESTA

def inverseOf(s: String): Option[Double] = 
    s.toIntOption flatMap 
    assertNonZero flatMap 
    inverse

In [ ]:
run(TestInverseOf(inverseOf))

# Ejercicio 2 (4 puntos)

Se desea implementar una función `collect` que combina el comportamiento de las funciones `map` y `filter`. Si la función `map` permite transformar una lista de `A`s en una lista de `B`s, la función `collect` permite aplicar esta transformación de manera selectiva, de tal forma que solamente algunos de los `A`s se transformen en `B`s. Para ello, además de la lista a transformar, `collect` recibe como argumento una función del tipo `A => Option[B]`; los elementos que se transformarán serán precisamente aquellos para los que dicha función devuelva un valor distinto de `None`. A continuación se muestran algunos ejemplos de uso de la función `collect` sobre listas de cadenas de caracteres:

In [ ]:
class TestCollect(collect: List[String] => (String => Option[Int]) => List[Int]) extends AnyFlatSpec with should.Matchers:

    "collect" should "work" in:
        collect(List("asdg", "12", "X", "652"))(_.toIntOption) shouldBe List(12, 652)
        collect(List("12", "652"))(_.toIntOption) shouldBe List(12, 652)
        collect(List("asdg", "X"))(_.toIntOption) shouldBe List()
        collect(List())(_.toIntOption) shouldBe List()
        

#### a) (1 punto)

Implementa la función `collect` de manera recursiva.

In [ ]:
// IMPLEMENTA TU RESPUESTA

def collect[A, B](l: List[A])(f: A => Option[B]): List[B] = 
    l match 
        case Nil => Nil
        case h :: t => 
            f(h) match 
                case None => collect(t)(f)
                case Some(e) => e :: collect(t)(f)

In [ ]:
run(TestCollect(collect))

#### b) (1 punto)

Implementa la función `collect` mediante la función `foldRight`.

In [ ]:
// IMPLEMENTA TU RESPUESTA

def collect[A, B](l: List[A])(f: A => Option[B]): List[B] = 
    l.foldRight(List[B]()): 
        (h, tailSol) => 
            f(h) match 
                case None => tailSol
                case Some(e) => e :: tailSol

In [ ]:
run(TestCollect(collect))

#### c) (1 punto)

Implementa la función `collect` mediante las funciones `map` y `filter`. Ténganse en cuenta para ello los métodos `isDefined ` y `get` de la clase `Option[_]` descritas en el preámbulo del examen.

In [ ]:
// IMPLEMENTA TU RESPUESTA

def collect[A, B](l: List[A])(f: A => Option[B]): List[B] = 
    l.map(f)
     .filter(_.isDefined)
     .map(_.get)

In [ ]:
run(TestCollect(collect))

#### d) (1 punto)

La función `lefts` recibe una lista de `A`s o `B`s y devuelve una lista formada únicamente por los valores de tipo `A`. A continuación se muestran algunos ejemplos de funcionamiento: 

In [ ]:
class TestLefts(lefts: List[Either[String, Int]] => List[String]) extends AnyFlatSpec with should.Matchers:

    "lefts" should "work" in:
        lefts(List(Left("a"), Left("b"), Right(3), Left("c"), Right(4))) shouldBe List("a", "b", "c")
        lefts(List(Left("a"), Left("b"), Left("c"))) shouldBe List("a", "b", "c")
        lefts(List(Right(3), Right(4))) shouldBe List()
        lefts(List()) shouldBe List()

Implementa la función `lefts` haciendo uso de la función `collect`.

In [ ]:
// IMPLEMENTA TU RESPUESTA

def lefts[A, B](l: List[Either[A, B]]): List[A] = 
    collect(l): 
        case Left(a) => Some(a)
        case Right(_) => None

In [ ]:
run(TestLefts(lefts))

# Ejercicio 3 (3 puntos)

Se desea implementar la función `corresponds` que, dadas dos listas y un predicado binario, devuelve cierto si todos los pares de elementos correspondientes de ambas listas satisfacen el predicado. Dos elementos son _correspondientes_ si se encuentran en la misma posición de sus listas. Por ejemplo, los siguientes casos de prueba se basan en un predicado `equals`, que recibe una cadena de caracteres y un número entero, y devuelve cierto si la representación numérica de la cadena es igual a dicho número:

In [ ]:
class TestCorresponds(corresponds: (List[Int], List[String]) => ((Int, String) => Boolean) => Boolean) 
extends AnyFlatSpec with should.Matchers:

    val equals: (Int, String) => Boolean = 
        (x, y) => x == y.toInt

    "corresponds" should "work" in:
        corresponds(List(), List())(equals) shouldBe true
        corresponds(List(1,2,3,4), List("1", "2", "3", "4"))(equals) shouldBe true
        corresponds(List(1,2,3,4), List("1", "22", "3", "4"))(equals) shouldBe false
        corresponds(List(1,2,3), List("1", "2", "3", "4"))(equals) shouldBe false
        corresponds(List(1,2,3,4), List("1", "2", "3"))(equals) shouldBe false
        

Como se observa en el segundo ejemplo, todos los pares correspondientes de ambas listas (`(1,"1")`, `(2, "2")`, etc.) satisfacen el predicado, por lo que el resultado es `true`. Asimismo, como se observa en los dos últimos ejemplos, para que dos listas sean correspondientes tienen que tener el mismo tamaño.

#### a) (1.5 puntos)

Implementa la función `corresponds` mediante recursión final. 

In [ ]:
// IMPLEMENTA TU RESPUESTA

def corresponds[A, B](l1: List[A], l2: List[B])(f: (A, B) => Boolean): Boolean = 
    (l1, l2) match 
        case (Nil, Nil) => true
        case (ha::ta, hb::tb) if f(ha, hb) => corresponds(ta, tb)(f)
        case _ => false

In [ ]:
run(TestCorresponds(corresponds))

#### b) (1.5 puntos)

Implementa la función `corresponds` mediante los métodos de la librería estándar de Scala de la clase `List[_]`, `size`, `zip` y `forall`. Se encontrará una descripción de estos métodos en el preámbulo del examen.

In [ ]:
// IMPLEMENTA TU RESPUESTA

def corresponds[A, B](l1: List[A], l2: List[B])(f: (A, B) => Boolean): Boolean = 
    l1.size == l2.size && 
    l1.zip(l2).forall(f.tupled)

In [ ]:
run(TestCorresponds(corresponds))